# Setup

In [ ]:
# from google.colab import userdata
# openai_api_key = userdata.get('ai_agents_openai')

In [ ]:
# !pip install 'crewai[tools]' -q

In [1]:
# Import libraries
# from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task, LLM
from IPython.display import display, Markdown
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from langchain_google_genai import ChatGoogleGenerativeAI
import dotenv

In [ ]:
# Set the API keys as environment variables
# os.environ['OPENAI_API_KEY'] = openai_api_key

# Load environment variables from .env file
dotenv.load_dotenv()
# Set the API keys as environment variables
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')


In [3]:
# Define the LLM
# llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0.5)

# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.5)
llm = LLM(
    model="gemini/gemini-2.5-flash-preview-04-17",
    temperature=0.5
)

llm.call("Hello, what model are you")

'I am a large language model, trained by Google.'

# Get the data

*   Interviewer
*   company
*   job position
*   job description



In [4]:
# Store the data
interviewer = input("Enter the name of the interviewer: ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

In [ ]:
print(f"Interviewer: {interviewer}")
print(f"Company: {company}")
print(f"Job Position: {job_position}")
print(f"Job Description: {job_description}")

# Interviewer AI Agent

In [18]:
# Create the interver AI agent
interviewer_agent = Agent(
    role = f" You are {interviewer}, employed at {company}, and hiring for job position {job_position}",
    goal = f" you help the user user prepare for the job interview for job {job_position} with the following description: {job_description}",
    backstory = """
    ## Experience

    Sensay
    Chief Human Resources Officer
    Full-time
    Mar 2024 - Present · 1 yr 2 mos
    Remote
    Scaling talent and building a world-class people experience at Sensay, fueling our rapid growth in the AI space.
    Talent Acquisition: Lead recruiting efforts to identify and attract top talent aligned with our mission.
    Employee Development: Implement training and development programs to enhance employee skills and career growth.
    Culture Building: Foster an inclusive and innovative workplace culture that supports our digital legacy mission.
    Performance Management: Oversee performance evaluation processes to ensure organizational goals are met.
    HR Strategy: Develop and execute HR strategies that align with business objectives and drive organizational success.
    Employee Engagement: Promote initiatives that enhance employee satisfaction and retention.
    Compliance: Ensure all HR practices comply with legal regulations and industry standards.
    Human Resources (HR), Talent Management

    KOLZ
    CHRO Advisor
    Contract
    Aug 2024 - Present · 9 mos
    Remote
    24/7 Chat With the Best Key Opinion Leaders Powered by AI
    Sensay Labs introduces a groundbreaking service that allows users to engage in 24/7 chat with digital replicas of key opinion leaders (KOLs) in various fields. This innovative platform leverages advanced AI technology to replicate the knowledge, personality, and communication style of these experts, providing users with personalized, consistent, and knowledgeable support around the clock

    MagicCraft Ltd.
    2 yrs 11 mos total
    Chief Human Resources Officer
    Freelance
    Nov 2023 - Jan 2025 · 1 yr 3 mos
    Remote
    Overseeing HR and TA departments and developing strategies and policies that promote sustainable growth
    Human Resources Business Partner
    Contract
    Feb 2023 - Nov 2023 · 10 mos
    Remote
    Head of HR/TA
    Driving the full spectrum of recruiting and selection activities.
    Creating and implementing talent acquisition strategies for leadership, mid-career and graduate hiring.
    Partnering with leaders within the organization to track the ongoing hiring needs of their business.
    Managing end-to-end recruitment.
    Staying up-to-date with industry trends, best practices, and emerging recruitment technologies, incorporating innovative approaches into our talent acquisition efforts.
    Developing the TA agenda.
    Developing the HR agenda.
    Developing HR strategies, policies, and practices.
    Improving and monitoring employee productivity.
    Structuring compensation and benefit packages.
    Managing staff wellness initiatives.
    Improving relations between staff and employers.
    Evaluating staffing needs.
    Overseeing recruitment efforts.
    Managing and allocating HR funds.
    Engaging with heads of department.
    Startup Environments, Cash Flow
    Talent Acquisition Manager
    Contract
    Jun 2022 - Feb 2023 · 9 mos
    Startup Environments, Cash Flow

    Chatgod.ai
    Human Resources Business Partner Consultant
    Contract
    Mar 2023 - Sep 2023 · 7 mos
    Remote
    ChatGod.ai is a Company Under the same Lab of MagicCraft Brand
    Startup Environments, Cash Flow

    Carrier Commercial Refrigeration
    Full-time · 1 yr 6 mos total
    Country Lead
    Apr 2022 - Nov 2022 · 8 mos
    Praga, Cechia
    EMEA HR manager, Leading a team of 8 HRs specialist for France/Benelux countries.
    Cash Flow, HR Analytics
    Senior Human Resources Specialist
    Jun 2021 - Apr 2022 · 11 mos
    Prague, Czechia
    """,
    llm = llm
)

In [19]:
# Define the interview prep task
interview_prep_task = Task(
    description = f"""
    Interview the user for the job {job_position} with the following description: {job_description}
    """,
    expected_output = f"""
    ask only one question to the user that is relevant for the job {job_position} that has not been asked before
    """,
    agent = interviewer_agent
)

Test Crew

In [ ]:
test_crew = Crew(
    agents=[interviewer_agent],
    tasks=[interview_prep_task],
    verbose=True,
    process=Process.sequential
)

result = test_crew.kickoff()
result

# AI Coach

In [7]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position} by grading the relevance of the answer",
    backstory = "You are an pro expert in AI Engineering job interviews",
    llm = llm)

In [12]:
# Define the Coaching Task
coaching_task = Task(
    description = """ Give feedback to the user on its answer """,
    expected_output = f""" Give feedback to the user on its answer with a bullet points on what was good, bad, and how to take it to the next level""",
    agent = coach_agent,
    context = [interview_prep_task],
    human_input=True
)

In [ ]:
test_crew1 = Crew(
    agents=[interviewer_agent, coach_agent],
    tasks = [interview_prep_task, coaching_task],
    verbose=True,
)

result = test_crew1.kickoff()

# AI Crew

In [ ]:
# Create an empty list for questions
interview_questions = []
# # Store each question
# interview_questions.append(result.tasks_output[0].raw)
# interview_content = ' '.join(interview_questions)
# string_source = StringKnowledgeSource(content = interview_content)


In [ ]:
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, coaching_task],
    verbose = True,
    process = Process.sequential,
    memory = True,
    # knowledge_sources = [string_source]
)
result = crew.kickoff()

# # Store each question
# interview_questions.append(result.tasks_output[0].raw)
# interview_content = ' '.join(interview_questions)
# string_source = StringKnowledgeSource(content = interview_content)

ValidationError: 1 validation error for Crew
  Value error, Please provide an OpenAI API key. You can get one at https://platform.openai.com/account/api-keys [type=value_error, input_value={'agents': [Agent(role= Y...ntial'>, 'memory': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# Check output
display(Markdown(result.tasks_output[0].raw))

In [20]:
# 1) run the interviewer
test_crew = Crew(
    agents  = [interviewer_agent],
    tasks   = [interview_prep_task],
    verbose = True,
    process = Process.sequential,
)
result1 = test_crew.kickoff()
question = result1.tasks_output[0].raw
display(Markdown(question))

# get your answer
user_answer = input("Your answer: ")

# 2) build a feedback Task, *using context* rather than human_input or memory
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

qa_source = StringKnowledgeSource(content=f"Q: {question}\nA: {user_answer}")

coaching_task = Task(
    description      = "Coach: give feedback to the answer above",
    expected_output  = "Bullet‑point feedback on what was good, what was weak, and how to improve",
    agent            = coach_agent,
    context          = [qa_source],            # ← inject Q&A here
    human_input      = False                   # ← ensure it doesn’t wait for another human turn
)

# run the coach
coach_crew = Crew(
    agents  = [coach_agent],
    tasks   = [coaching_task],
    verbose = True,
    process = Process.sequential,
)
result2 = coach_crew.kickoff()
display(Markdown(result2.tasks_output[0].raw))

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0aededa8-cac3-4835-84d2-1384ceb90148                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 2424132f-0697-41e2-a73a-23c13017a7cd
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2424132f-0697-41e2-a73a-23c13017a7cd
       Status: Executing Task...
    └── 🤖 Agent:  You are  Alessandro Huaroto , employed at Sensay, and hiring for job position Artificial 
        Intelligence Engineer
            Status: In Progress

# Agent:  You are  Alessandro Huaroto , employed at Sensay, and hiring for job position Artificial Intelligence Engineer
## Task: 
    Interview the user for the job Artificial Intelligence Engineer with the following description: About the job At Sensay, we're on a mission to shape the future of AI personal replicas. As we expand our development team, we are looking for a talented AI Developer to help build outstanding features that our customers love.   Interact with Our Founder Replica: Get to know our vision by interacting with our founder replica, Dan, here.   About the Role: As an AI Developer, you will be responsible for designing, developing, and maintaining our AI systems using cutting-edge frameworks like Next.js, Supabase, and TypeScript. You will collaborate closely with product owners, UX specialists, and other developers to deliver high-quality AI solutions that meet the needs of our users.   What You'll Do: Design and Implement AI Solutions: Utilize your AI expertise to 

🚀 Crew: crew
└── 📋 Task: 2424132f-0697-41e2-a73a-23c13017a7cd
       Status: Executing Task...
    └── 🤖 Agent:  You are  Alessandro Huaroto , employed at Sensay, and hiring for job position Artificial 
        Intelligence Engineer
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 2424132f-0697-41e2-a73a-23c13017a7cd
       Assigned to:  You are  Alessandro Huaroto , employed at Sensay, and hiring for job position Artificial 
    Intelligence Engineer
       Status: ✅ Completed
    └── 🤖 Agent:  You are  Alessandro Huaroto , employed at Sensay, and hiring for job position Artificial 
        Intelligence Engineer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2424132f-0697-41e2-a73a-23c13017a7cd                                                                     │
│  Agent:  You are  Alessandro Huaroto , employed at Sensay, and hiring for job position Artificial Intelligence  │
│  Engineer                                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0aededa8-cac3-4835-84d2-1384ceb90148                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Hello, thanks for your interest in the Artificial Intelligence Engineer position at Sensay. As you know, we're building the future of AI personal replicas, which requires robust and scalable systems. Could you describe a challenging AI project you've worked on where designing for scalability was a critical factor, and how you approached that challenge from implementation to deployment?

AttributeError: 'StringKnowledgeSource' object has no attribute 'get'